# Setup

In [11]:
import os

from dotenv import load_dotenv

import fixpoint
from fixpoint.agents.openai import OpenAIClients
from fixpoint.agents.protocol import TikTokenLogger
from fixpoint.memory import memgpt_summarize, MemGPTSummarizeOpts, MemGPTSummaryAgent, Memory
from fixpoint.utils.messages import umsg, amsg, smsg

In [4]:
import logging
import sys
from fixpoint.logging import logger
logging.basicConfig(stream=sys.stdout)
logger.setLevel(logging.INFO)
logger.info("Info mode is on?")

INFO:fixpoint:Info mode is on?


In [5]:
assert load_dotenv('../.env')

In [13]:
model_name = 'gpt-3.5-turbo'
agent_mem = Memory()
tokenlogger = TikTokenLogger(model_name)

# Use an OpenAI compatible agent

In [15]:
agent = fixpoint.agents.OpenAI(
    model_name=model_name,
    openai_clients=OpenAIClients.from_api_key(os.environ['OPENAI_API_KEY']),
    memory=agent_mem,
    pre_completion_fns=[tokenlogger.tiktoken_logger]
)

In [16]:
completion = agent.chat.completions.create(
    messages=[{"role": "user", "content": "What is a fixpoint, in math terms?"}]
)

INFO:fixpoint:Total input tokens: 12


In [20]:
print(completion.choices[0].message.content)

In mathematics, a fixpoint refers to a value that remains unchanged when a particular function or operation is applied to it. In other words, a fixpoint is a point or element within a set that is mapped to itself under a given transformation. Fixpoints are important in various branches of mathematics, such as calculus, topology, and dynamical systems.


In [27]:
print("Number of memories:", len(agent_mem.memory()))

Number of memories: 1


In [28]:
print(agent_mem.to_str())

user: What is a fixpoint, in math terms?
assistant: In mathematics, a fixpoint refers to a value that remains unchanged when a particular function or operation is applied to it. In other words, a fixpoint is a point or element within a set that is mapped to itself under a given transformation. Fixpoints are important in various branches of mathematics, such as calculus, topology, and dynamical systems.


# Use a memory summarizer agent

In [ ]:
agent = fixpoint.agents.OpenAIAgent(
    model_name=model_name,
    openai_clients=OpenAIClients.from_api_key(os.environ['OPENAI_API_KEY']),
    memory=agent_mem,
    pre_completion_fns=[tokenlogger.tiktoken_logger]
)

In [8]:
memgpt_sum_opts = MemGPTSummarizeOpts(
    agent=agent,
    context_window=200
)

In [12]:
summary_agent = MemGPTSummaryAgent(memgpt_sum_opts)

In [ ]:
compl = summary_agent.create_completion(messages=[
    smsg("You are a friendly AI customer support agent. You chat with a user to answer their questions"),
    umsg("What is your name?")
])

In [ ]:
agent_mem.memory()[0][0]

In [ ]:
agent_mem.memory()

In [ ]:
compl = summary_agent.create_completion(messages=[
    smsg("You are a friendly AI customer support agent. You chat with a user to answer their questions"),
    umsg("Can you help me pick out a new sports car?")
])

In [ ]:
agent_mem.memory()

In [ ]:
compl = summary_agent.create_completion(messages=[
    smsg("You are a friendly AI customer support agent. You chat with a user to answer their questions"),
    umsg("hi")
])

In [ ]:
agent_mem.memory()[-1]

In [ ]:
print(agent_mem.format_str())